In [1]:
# from autogenstudio.src.autogenstudio.workflowmanager import WorkflowManager

In [11]:
import asyncio
from typing import Sequence

from autogen_agentchat.base import BaseChatAgent
from autogen_agentchat.teams import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import (
    ChatMessage,
    StopMessage,
    TextMessage,
)
from autogen_core.base import CancellationToken


class CustomAgent(BaseChatAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name, "A human user.")

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> ChatMessage: 
        return TextMessage(content="Hello world", source=self.name)

custom_agent = CustomAgent("User")
team = RoundRobinGroupChat([ custom_agent])
round_robin_team_result = await team.run(
    "Write a Haiku about the weather in Paris", termination_condition=MaxMessageTermination(max_messages=2)
) 


--------------------------------------------------------------------------- 
[2024-10-25T11:19:16.206489]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-10-25T11:19:16.207659], User:

Hello world
--------------------------------------------------------------------------- 
[2024-10-25T11:19:16.209106], User:

Hello world
--------------------------------------------------------------------------- 
[2024-10-25T11:19:16.209419], Termination:

Maximal number of messages 2 reached, current message count: 2

In [14]:
round_robin_team_result

TaskResult(messages=[TextMessage(source='user', content='Write a Haiku about the weather in Paris'), TextMessage(source='User', content='Hello world'), TextMessage(source='User', content='Hello world')])

In [ ]:
class Provider():
    def __init__(self):
        self.workflow_manager = WorkflowManager()

    async def run(self, message: str):
        return await self.workflow_manager.run(message)